In [ ]:
using<-function(...) {
    libs<-unlist(list(...))
    req<-unlist(lapply(libs,require,character.only=TRUE))
    need<-libs[req==FALSE]
    if(length(need)>0){ 
        install.packages(need)
        lapply(need,require,character.only=TRUE)
    }
}

In [ ]:
using("data.table", "tidyverse", "naniar", "stringr", "readr",  "dplyr", "magrittr", "readxl", "writexl", "sjmisc", "tidyr", "rdflib"
                  )

In [ ]:
# Function to apply SHA-256 hashing


sha256_hash <- function(data) {
  openssl::sha256(data)
}

In [ ]:
## Data loading



barometer_dt_raw_2021 <- readxl::read_excel("Data/Lab5/2021_Final_Anonymised_data_Only_2023-04-20.v2.xlsx")
barometer_dt_raw_2022 <- readxl::read_excel("Data/Lab5/2022_Final_Anonymised_data_Only_2023-04-21.xlsx")


In [ ]:
## Add new dataset

barometer_dt_combined <- rbind(barometer_dt_raw_2021, barometer_dt_raw_2022)


In [ ]:
## Filter data 

barometer_dt_filter <- barometer_dt_combined %>% 
  dplyr::filter(SYSTEM %in% c('Respiratory', 'NA')
                )
barometer_dt_filter2 <- barometer_dt_filter %>% 
  dplyr::filter(ALIQUOTMATRIXTYPE %in% c('Pleural Fluid', 'Tissue swab', 'Tonsil',  'Lymph Node - Multiple',  'Trachea',  'Thoracic Fluid', 'Lung', 'Swab', 'Culture',  'Thymus', 'Part Carcass', 'Tissue swab', 'Nasal Swab', 'Nasal Fluid', 'Tissue-Pool', 'Tissue (VTM)', 'Carcass', 'Lymph Node', 'Pooled swab', 'Misc.')
                ) 
barometer_dt_filter3 <- barometer_dt_filter2 %>% 
  dplyr::filter(TEST %in% c("PI3V PCR", "PCR M. haemolytica - ARVL", "Mycoplasma bovis (PCR)", 
                            "PCR H. somni - ARVL", "PCR P. multocida - ARVL", "Miscellaneous Test",
                            "Routine Culture", "PCR M. bovis - ARVL", "BRSV PCR", 
                            "Culture Growth", "PCR BoCoV", "Mycoplasma bovis (PCR)")
) 




In [ ]:
## Data manipulation

barometer_dt <- barometer_dt_filter3 %>% 
  dplyr::rename(
    Filenumber = SDGa,
    Samplenumber = SAMPLEa,
    Farm_ID = HERD_NOa,
    Date = DELIVERY_DATE,
    breed = Herd.Type
    ) %>%
  dplyr::mutate(
    Country ='Ireland',
    Lab_reference ='5', 
    Sample_type = case_when(
      SUBCLASS == 'Carcass' ~ 'Autopsy', 
      ALIQUOTMATRIXTYPE %in% c('Carcass', 'Lung', 'Thymus', 
                               'Lymph Node - Multiple', 'Tissue-Pool', 
                               'Lymph Node', 'Tissue (VTM)', 
                               'Part Carcass') ~ 'Autopsy',
      ALIQUOTMATRIXTYPE %in% c('Swab', 'Nasal Swab', 'Pooled swab', 
                               'Nasal Fluid') ~ 'Swab', 
      ALIQUOTMATRIXTYPE %in% c('Trachea', 'Thoracic Fluid', 'Culture', 
                               'Fluid', 'Misc.', 'Pleural Fluid') ~ 'Unknown',
      TRUE ~ 'Missing' 
    ),
    Diagnostic_test = case_when(
      TEST %in% c("PI3V PCR", "PCR M. haemolytica - ARVL", 
                  "Mycoplasma bovis (PCR)", "PCR H. somni - ARVL",
                  "PCR M. bovis - ARVL", "BRSV PCR", "PCR BoCoV", 
                  "Mycoplasma bovis (PCR)", "PCR P. multocida - ARVL") ~ 'PCR',
      TEST %in% c("Routine Culture", "Culture Growth") ~ "Culture",
      #TEST == 'Maldi ToF' ~ 'MALDI-TOF',
      #TEST == "Next Generation Sequencing" ~ 'NGS',
      #TEST == "Miscellaneous Test" ~ 'Unknown', 
      ## Otherwise 'Unknown' without results in final format (could also be removed?)
      TRUE ~ 'Missing'
    ), 
    Breed = case_when(
      breed == "BEEF" ~ 'Beef',  
      breed == "DAIRY" ~'Dairy',
      breed == "SUCKLER" ~ 'Suckler', ## not sure to add another production system or 'unknown'/'other? 
      breed %in% c("OTHER") ~ 'Unknown',
      TRUE ~ 'Unknown' 
    ),
    Province = County,
    Pathogen = case_when(
      TEST == "PCR P. multocida - ARVL" ~ 'PM',
      TEST == "PCR M. haemolytica - ARVL" ~ 'MH',
      TEST %in% c("PCR H. somni - ARVL", "H. somni PCR") ~ 'HS',
      TEST %in% c("Mycoplasma bovis (PCR)", "PCR M. bovis - ARVL") ~ 'MB',
      TEST == "PI3V PCR" ~ 'PI3',
      TEST == "PCR BoCoV" ~ 'BCV',
      TEST == "BRSV PCR" ~ 'BRSV',
      TRUE ~ 'Missing'
    ),
  )%>%
  dplyr::select(
    Filenumber,
    Samplenumber,
    Diagnostic_test,
    Country,
    Lab_reference,
    Sample_type,
    Breed,
    Pathogen,
    Date,
    Province,
    RESULT,
    RESULTNAME,
    AGENT,
    Farm_ID
  ) %>%
  dplyr::distinct()%>%
  dplyr::mutate(
    Filenumber= sha256_hash(as.character(Filenumber)),
    Samplenumber= sha256_hash(as.character(Samplenumber)),
    Farm_ID= sha256_hash(as.character(Samplenumber))
  ) 




In [ ]:
## Toevoegen extra rijen voor cultuur (& MALDI & NGS?)

barometer_dt$HS <- ifelse(barometer_dt$Diagnostic_test == "Culture", 0, NA)
barometer_dt$MH <- ifelse(barometer_dt$Diagnostic_test == "Culture", 0, NA)
barometer_dt$PM <- ifelse(barometer_dt$Diagnostic_test == "Culture", 0, NA)

barometer_dt_culture_wide <- barometer_dt %>%
  tidyr::pivot_longer(
    cols = c('PM', 'MH', 'HS'),
    names_to = 'Pathogen_culture',
    values_to = 'Result_culture'
  )

barometer_dt_culture_wide$Pathogen <- 
  ifelse(barometer_dt_culture_wide$Pathogen == "Missing",
         barometer_dt_culture_wide$Pathogen_culture,
         barometer_dt_culture_wide$Pathogen)


## Create binary results PCR & culture

barometer_results <- barometer_dt_culture_wide %>% 
  dplyr::mutate(
    Result = case_when(
      # Positive tests
      Diagnostic_test == 'PCR' & 
        RESULT %in% c("Positive", "Weak Positive", "Mycoplasma bovis PCR Positive",
                      "Strong Positive") ~ 1,
      # Negative tests
      Diagnostic_test == 'PCR' & 
        RESULT %in% c("No Pathogen detected", "Negative", "Sterile", 
                      "No Significant Growth", "No CT", 
                      "Mycoplasma bovis PCR Negative", 
                      "Mixed Non-Significant Bacterial Growth", 
                      "No Significant Growth @48hrs", "No Growth", 
                      "No Pathogen detectedn", "No RNA detected", 
                      "No DNA detected", "No Virus Detected", "Not Detected") ~ 0,
      # Inconclusive tests
      Diagnostic_test == 'PCR' & 
        RESULT %in% c("Inconclusive", "Mixed Bacterial Growth", "Mixed Growth",
                      "Very Mixed Growth") ~ as.numeric(NA),
      # Culture tests
      Diagnostic_test == 'Culture' & Pathogen %in% c('MH', 'PM', 'HS') ~ 
        case_when(
          Pathogen == 'MH' & RESULT == "Mannheimia haemolytica" ~ 1,
          Pathogen == 'PM' & RESULT %in% c("Pasteurella multocida", 
                                           "P. multocida") ~ 1,
          Pathogen == 'HS' & RESULT %in% c("Histophilus somni",
                                           "Histophilus somnus",
                                           "Histophilus somnii") ~ 1,
          # Default case
          TRUE ~ 0
        ),
    )
  ) %>%
  dplyr::select(
    Filenumber,
    Samplenumber,
    Diagnostic_test,
    Country,
    Lab_reference,
    Sample_type,
    Breed,
    Pathogen,
    Result,
    Date,
    Province,
    RESULT,
    RESULTNAME,
    AGENT,
    Farm_ID
  ) %>%
  dplyr::distinct()%>%
  dplyr::mutate(
    Filenumber= sha256_hash(as.character(Filenumber)),
    Samplenumber= sha256_hash(as.character(Samplenumber)),
    Farm_ID= sha256_hash(as.character(Samplenumber))
  )



In [ ]:
## Floor date to 1st of month

barometer_results$Floored_date <- lubridate::floor_date(barometer_results$Date, "month")

## Aggregate data based on farm_ID & month

barometer_groupby <- barometer_results %>%
  group_by(Lab_reference, Country, Breed, Floored_date, Province,
           Farm_ID, Diagnostic_test, Sample_type, Pathogen) %>%
  summarise(across(c(Result), ~ ifelse(all(is.na(.)), NA, max(., na.rm = TRUE))))

# If all are NA, than NA, if not (else): max in group, while ignoring NA 



In [ ]:
library(magrittr)
library(rdflib)
library(XML)

In [ ]:
  
    # create an RDF object
Literal <- function(value, datatype="xsd:string") {
  rdf_literal(value, datatype=datatype)
}
rdf <- rdf()
 
# create a namespace prefix for the ontology
onto <- "http://www.purl.org/decide/LivestockHealthOnto"
 
# iterate through each row in the barometer DataFrame
for (i in 1:nrow(barometer_groupby)) {
  row <- barometer_groupby[i,] 
# create a URI for the CattleSample individual based on the row ID
CattleSample <- paste0(onto, "CattleSample", i)

# add triples to the RDF graph


  #rdf_add(rdf, CattleSample, paste0(prefix, "hasBreed"), as.character(row[3]), datatype="xsd:string")
  rdf_add(rdf, CattleSample, paste0(onto, "hasLabReference"), ifelse(is.na(row$Lab_reference), "", as.character(row$Lab_reference)))
  rdf_add(rdf, CattleSample, paste0(onto, "hasCountry"), ifelse(is.na(row$Country), "", as.character(row$Country)))
  rdf_add(rdf, CattleSample, paste0(onto, "hasBreed"), ifelse(is.na(row$Breed), "", as.character(row$Breed)), datatype="xsd:string")
  rdf_add(rdf, CattleSample, paste0(onto, "hasDate"), ifelse(is.na(row$Floored_date), "", as.character(row$Floored_date)), datatype="xsd:string")
  rdf_add(rdf, CattleSample, paste0(onto, "hasProvince"), ifelse(is.na(row$Province), "", as.character(row$Province)), datatype="xsd:string")
  rdf_add(rdf, CattleSample, paste0(onto, "hasFarmIdentification"), ifelse(is.na(row$Farm_ID), "", as.character(row$Farm_ID)), datatype="xsd:string")
  rdf_add(rdf, CattleSample, paste0(onto, "hasDiagnosticTest"), ifelse(is.na(row$Diagnostic_test), "", as.character(row$Diagnostic_test)), datatype="xsd:string")
  rdf_add(rdf, CattleSample, paste0(onto, "hasSampleType"), ifelse(is.na(row$Sample_type), "", as.character(row$Sample_type)), datatype="xsd:string")
  rdf_add(rdf, CattleSample, paste0(onto, "hasPathogen"), ifelse(is.na(row$Pathogen), "", as.character(row$Pathogen)), datatype="xsd:string")
  rdf_add(rdf, CattleSample, paste0(onto, "hasResult"), ifelse(is.na(row$Result), "Missing", as.character(row$Result)), datatype="xsd:string")

}
 
# print the RDF graph (for testing)
cat(rdf_serialize(rdf, format="turtle"))
 
# output the RDF graph to a file
## Save file to RDF (Long Version) instead of CSV
rdf_serialize(rdf, "output/RDFoutputCattleSampleLab5.ttl", format="turtle")